# Basis and reconstruction of 64 pixel image.
Joshua Stough
DIP

Decompose and partially reconstruct according to the Haar
basis.



In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from waveletUtil import *
from ipywidgets import VBox, FloatSlider

In [2]:
print('haarReconstructionDemo: Works only on 64 pixel images...')

# I = plt.imread('surprise.png')
I = plt.imread('hello.png')
I = I[...,:3].copy()

haarReconstructionDemo: Works only on 64 pixel images...


In [3]:
from skimage.transform import rescale
plt.figure()
sI = rescale(I, 1/16, order = 1, anti_aliasing=False, multichannel=True)
plt.imshow(sI)
# sI = I.copy()
sI.shape

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(32, 32, 3)

In [4]:
s = 32

In [8]:
# H is the 8x8 Haar matrix
# H = np.eye(8) # For fun comparison.
# H = makeRandomBasis(8)
# H = makeKLTBasis(I, 8)
H = makeDCTMatrix(s)
# H = makeHaarMatrix(s)

In [9]:
#The transform image, an image of coefficients wrt the Haar basis.
CT = np.zeros(sI.shape)
for chan in range(3):
    CT[...,chan] = np.matmul(H, np.matmul(sI[...,chan], H.transpose()))


#RI will represent the reconstructed image as we add back more
#Haar patterns
RI = np.zeros(sI.shape) #Still should be 8x8
sI.shape

(32, 32, 3)

In [10]:
# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(s), np.arange(s), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
dists = np.sum(coords*coords, axis=1)
darg = np.argsort(dists) #sorts in increasing order
darg.min()

0

In [11]:
from ipywidgets import VBox, IntSlider

In [12]:
#Visual
fh, axh = plt.subplots(s,s, figsize=(8,8))
fr, axr = plt.subplots(s,s, figsize=(8,8))


# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(s), np.arange(s), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
dists = np.sum(coords*coords, axis=1)
darg = np.argsort(dists) #sorts in increasing order

#If we were to use magnitude of the coefficient...need to ravel in
#column-major, like we set up coords.
# mags = T.ravel(order='F')
# darg = np.argsort(mags)
# darg.reverse()
#
#c = 1 #used to display the order of reconstruction.

for ind in darg:
    i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
    Bij = np.outer(H[i, :], H[j, :])
    axh[i][j].imshow(Bij, cmap='gray', vmin=-.5, vmax=.5)
    axh[i][j].axes.get_xaxis().set_visible(False)
    axh[i][j].axes.get_yaxis().set_visible(False)
    # axh[i][j].set_title('c_%d_%d: %6.3f' % (i, j, T[i,j]))
    # https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.text
    axh[i][j].text(0, 6, r'$\pi:%6.3f$' % CT[i, j, :].sum(), fontsize=6, color='cyan')

    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
    for chan in range(3):
        RI[...,chan] += CT[i,j,chan]*Bij

    RIcopy = RI.copy().clip(0,1)
    axr[i][j].imshow(RIcopy)
    axr[i][j].axes.get_xaxis().set_visible(False)
    axr[i][j].axes.get_yaxis().set_visible(False)
    # axr[i][j].text(0, 6, '%d' % c, fontsize=10, color='cyan')
    #The order of reconstruction isn't that informative, but useful for debugging.
    #c += 1


plt.show()

fh.canvas.set_window_title('Haar Basis Images and Coefficients')
fh.tight_layout()

fr.canvas.set_window_title('Partial Reconstructions from Large Scale to Small')
fr.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/remote/anaconda-3.7-2020-01-14/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations


KeyboardInterrupt: 

In [13]:
def make_image(change):
    global coords, H, CT
    ReI = np.zeros(sI.shape)
    Bij = np.outer(H[0, :], H[0, :])
    for ind in range(change):
        i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
        Bij = np.outer(H[i, :], H[j, :])
    
    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
        for chan in range(3):
            ReI[...,chan] += CT[i,j,chan]*Bij
    return Bij, ReI

In [14]:
base, i = make_image(3)
base

array([[0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137],
       [0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137],
       [0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137],
       ...,
       [0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137],
       [0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137],
       [0.04398137, 0.04229119, 0.03897578, ..., 0.03897578, 0.04229119,
        0.04398137]])

In [18]:
basis, init = make_image(1)

plt.ioff()
plt.clf()

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#FloatLogSlider
slider = IntSlider(
    orientation='horizontal',
    value=0,
    min=0,
    max=s**2,
    step=1,
    description='Base element'
)

fig_args = {'num':' ', 'frameon':True}
fig, ax = plt.subplots(1,2, figsize=(10,3), **fig_args) 

# display artists I'll update
adisp = ax[0].imshow(basis, cmap='gray', vmin=-0.5, vmax=0.5)
hdisp = ax[1].imshow(init, cmap='magma', vmin=0, vmax=1)

def update_image(change):    
    new_base, new_re = make_image(change.new)
    hdisp.set_array(new_re)
    # need to reset the color limits each time since gaus_h is changing a lot. 
    hdisp.set_clim(0,1) 
    
    adisp.set_array(new_base)
    
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])